In [24]:
import re


teststr1 = '''
```tag1
content1
```
'''


teststr2 = '''
```tag1

```tag1
content1
```

```

```tag1
content2

```
'''


teststr3 = '''
```tag1

```tag2
content1
```

```tag1
content2
```tag1
content3
```
```

```

```tag2
content2

```
'''


In [36]:
import re


def find_level1_blocks(text):
    # Regular expressions for opening and closing patterns
    opening_pattern = r'```[^\s]+'  # Matches any pattern like ```xxx followed by non-whitespace characters
    closing_pattern = r'```(?=\s|$)'  # Matches standalone closing patterns, followed by space, newline, or end of string

    # Finding all opening and closing positions
    open_positions = [(m.start(), m.group()) for m in re.finditer(opening_pattern, text)]
    close_positions = [m.start() for m in re.finditer(closing_pattern, text)]

    matches = []
    open_stack = []
    nesting_level = 0

    i, j = 0, 0
    last_match_end = -1

    while i < len(open_positions) or j < len(close_positions):
        if i < len(open_positions) and (j >= len(close_positions) or open_positions[i][0] < close_positions[j]):
            # Handle an opening pattern
            open_stack.append(open_positions[i])
            nesting_level += 1
            i += 1
        else:
            # Handle a closing pattern
            if open_stack:
                start_pos, start_tag = open_stack.pop()
                nesting_level -= 1
                # If we're back to level 0, it's a level 1 match
                if nesting_level == 0:
                    match_start = start_pos
                    match_end = close_positions[j] + len('```')
                    if match_start > last_match_end:
                        matches.append((match_start, match_end))
                        last_match_end = match_end
            j += 1

    # Extract the substrings corresponding to the level 1 matches
    result = [text[start:end] for start, end in matches]
    return result


def block_finder(raw_text:str,block_tag:str):
   blocks = find_level1_blocks(raw_text)
   matches = [block[len(f'```{block_tag}'):-3].strip() for block in blocks if block.startswith(f'```{block_tag}')]
   return matches


print(block_finder(teststr3,'tag1'))

['```tag2\ncontent1\n```\n\n```tag1\ncontent2\n```tag1\ncontent3\n```\n```']
